## MRI Datasets

### Getting Data

#### Datalad

`Datalad` installs the data - which for a dataset means that we get the "small" data (i.e. the text files) and the download instructions for the larger files. We can now navigate the dataset like its a file system and plan our analysis.

`datalad install ///openfmri/ds000030`

Download a single subject's imaging files.

```
cd ds000030
datalad get sub-10159
```

Getting and dropping data

#### Amazon Web Services (AWS)

download T1w scans
```
!cat ../download_list | \
    xargs -I '{}' aws s3 sync --no-sign-request \
    s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/{}/anat \
    ../data/ds000030/{}/anat
```

download resting state fMRI scans
```
!cat ../download_list | \
    xargs -I '{}' aws s3 sync --no-sign-request \
    s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/{}/func \
    ../data/ds000030/{}/func \
    --exclude '*' \
    --include '*task-rest_bold*'
```

download fmriprep preprocessed anat data
```
!cat ../download_list | \
    xargs -I '{}' aws s3 sync --no-sign-request \
    s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{}/anat \
    ../data/ds000030/derivatives/fmriprep/{}/anat
```

download fmriprep preprocessed func data
```
!cat ../download_list | \
    xargs -I '{}' aws s3 sync --no-sign-request \
    s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{}/func \
    ../data/ds000030/derivatives/fmriprep/{}/func \
    --exclude '*' \
    --include '*task-rest_bold*'
```

### Tutorial Dataset

For this tutorial, we will be using a subset of a publicly available dataset, ds000030, from [openneuro.org](https://openneuro.org/datasets/ds000030). The dataset is structured according to the Brain Imaging Data Structure ([BIDS](https://bids-specification.readthedocs.io/en/stable/)). BIDS is a simple and intuitive way to organize and describe your neuroimaging and behavioural data. Neuroimaging experiments result in complicated data that can be arranged in several different ways. BIDS tackles this problem by suggesting a new standard (based on consensus from multiple researchers across the world) for the arrangement of neuroimaging datasets. Using the same organizational standard for *all* of your studies will also allow you to easily reuse your scripts and share data and code with other researchers.

Below is a tree diagram showing the folder structure of single MR session within ds000030. This was obtained by using the bash command `tree`.  
`!tree ../data/ds000030`

```
ds000030
├── CHANGES
├── dataset_description.json
├── derivatives
│   └── fmriprep
├── participants.tsv
├── README
├── sub-50083
│   ├── anat
│   │   ├── sub-50083_T1w.json
│   │   └── sub-50083_T1w.nii.gz
│   └── func
│       ├── sub-50083_task-rest_bold.json
│       └── sub-50083_task-rest_bold.nii.gz
└── task-rest_bold.json
```

The `participants.tsv` file is meant to describe some demographic information on each participant within your study (eg. age, handedness, sex, etc.) Let's take a look at the `participants.tsv` file to see what's been included in this dataset.

In order to load the data into Python, we'll need to import the `pandas` package. The `pandas` **dataframe** is Python's equivalent to an Excel spreadsheet.

In [ ]:
import pandas as pd

Error: Error: Activating Python 3.7.3 64-bit ('dmri_venv': pyenv) to run Jupyter failed with Error: StdErr from ShellExec, pyenv: activate: command not found
.

We'll use the `read_csv()` function. It requires us to specify the name of the file we want to import and the separator that is used to distinguish each column in our file (`\t` since we're working with a `.tsv` file).

In [2]:
participant_metadata = pd.read_csv('../data/ds000030/participants.tsv', sep='\t')

In order to get a glimpse of our data, we'll use the `head()` function. By default, `head` prints the first 5 rows of our dataframe.

In [ ]:
participant_metadata.head()

Error: Error: Activating Python 3.7.3 64-bit ('dmri_venv': pyenv) to run Jupyter failed with Error: StdErr from ShellExec, pyenv: activate: command not found
.

We can view any number of rows by specifying `n=?` as an argument within `head()`.  
If we want to select particular rows within the dataframe, we can use the `loc[]` function and identify the rows we want based on their index label (the numbers in the left-most column).

In [4]:
participant_metadata.loc[[6, 10, 12]]

participant_id diagnosis  age gender  bart  bht  dwi  pamenc  pamret  rest  \
6       sub-10225   CONTROL   35      M   1.0  1.0  1.0     NaN     NaN   1.0   
10      sub-10249   CONTROL   28      M   1.0  1.0  1.0     NaN     NaN   1.0   
12      sub-10271   CONTROL   41      F   1.0  1.0  1.0     NaN     NaN   1.0   

    scap  stopsignal  T1w  taskswitch  ScannerSerialNumber ghost_NoGhost  
6    1.0         1.0  1.0         1.0              35343.0      No_ghost  
10   1.0         1.0  1.0         1.0              35343.0      No_ghost  
12   1.0         1.0  1.0         1.0              35343.0      No_ghost

**EXERCISE**: Select the first 5 rows of the dataframe using `loc[]`.

In [5]:
participant_metadata.loc[:4]

participant_id diagnosis  age gender  bart  bht  dwi  pamenc  pamret  rest  \
0      sub-10159   CONTROL   30      F   1.0  NaN  1.0     NaN     NaN   1.0   
1      sub-10171   CONTROL   24      M   1.0  1.0  1.0     NaN     NaN   1.0   
2      sub-10189   CONTROL   49      M   1.0  NaN  1.0     NaN     NaN   1.0   
3      sub-10193   CONTROL   40      M   1.0  NaN  1.0     NaN     NaN   NaN   
4      sub-10206   CONTROL   21      M   1.0  NaN  1.0     NaN     NaN   1.0   

   scap  stopsignal  T1w  taskswitch  ScannerSerialNumber ghost_NoGhost  
0   1.0         1.0  1.0         1.0              35343.0      No_ghost  
1   1.0         1.0  1.0         1.0              35343.0      No_ghost  
2   1.0         1.0  1.0         1.0              35343.0      No_ghost  
3   NaN         NaN  1.0         NaN              35343.0      No_ghost  
4   1.0         1.0  1.0         1.0              35343.0      No_ghost

**EXERCISE:** How many participants do we have in total?

In [6]:
participant_metadata.shape

(272, 16)

There are 2 different methods of selecting columns in a dataframe:  
*  participant_metadata[`'<column_name>'`] (this is similar to selecting a key in a Python dictionary)  
*  participant_metadata.`<column_name>`  

Another way to see how many participants are in the study is to select the `participant_id` column and use the `count()` function.

In [7]:
participant_metadata['participant_id'].count()

272

**EXERCISE:** Which diagnosis groups are part of the study?  
*Hint: use the* `unique()` *function.*

In [8]:
participant_metadata['diagnosis'].unique()

array(['CONTROL', 'SCHZ', 'BIPOLAR', 'ADHD'], dtype=object)

If we want to count the number of participants in each diagnosis group, we can use the `value_counts()` function.

In [9]:
participant_metadata['diagnosis'].value_counts()

CONTROL    130
SCHZ        50
BIPOLAR     49
ADHD        43
Name: diagnosis, dtype: int64

**EXERCISE:** How many males and females are in the study? How many are in each diagnosis group?

In [10]:
participant_metadata['gender'].value_counts()

M    155
F    117
Name: gender, dtype: int64

In [11]:
participant_metadata.groupby(['diagnosis', 'gender']).size()

diagnosis  gender
ADHD       F         22
           M         21
BIPOLAR    F         21
           M         28
CONTROL    F         62
           M         68
SCHZ       F         12
           M         38
dtype: int64

When looking at the participant dataframe, we noticed that there is a column called `ghost_NoGhost`. We should look at the README file that comes with the dataset to find out more about this.

In [12]:
!cat ../data/ds000030/README

## UCLA Consortium for Neuropsychiatric Phenomics LA5c Study

Preprocessed data described in


Gorgolewski KJ, Durnez J and Poldrack RA. Preprocessed Consortium for Neuropsychiatric Phenomics dataset. F1000Research 2017, 6:1262
https://doi.org/10.12688/f1000research.11964.2


are available at https://legacy.openfmri.org/dataset/ds000030/ and via Amazon Web Services S3 protocol at: s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/

## Subjects / Participants
The participants.tsv file contains subject IDs with demographic informations as well as an inventory of the scans that are included for each subject.

## Dataset Derivatives (/derivatives)
The /derivaties folder contains summary information that reflects the data and its contents:

1. Final_Scan_Count.pdf - Plot showing the over all scan inclusion, for quick reference.
2. parameter_plots/ - Folder contains many scan parameters plotted over time. Plot symbols are color coded by imaging site. Intended to provide a gene

For this tutorial, we're just going to work with participants that are either CONTROL or SCHZ (`diagnosis`) and have both a T1w (`T1w == 1`) and rest (`rest == 1`) scan. Also, we'll only use data without a ghosting artifact in the T1w scan (`ghost_NoGhost == 'No_ghost'`).

<b>EXERCISE:</b> Filter <code>participant_metadata</code> so that only the above conditions are present.

In [13]:
participant_metadata = participant_metadata[(participant_metadata.diagnosis.isin(['CONTROL', 'SCHZ'])) & 
                                            (participant_metadata.T1w == 1) & 
                                            (participant_metadata.rest == 1) & 
                                            (participant_metadata.ghost_NoGhost == 'No_ghost')]
participant_metadata

participant_id diagnosis  age gender  bart  bht  dwi  pamenc  pamret  \
0        sub-10159   CONTROL   30      F   1.0  NaN  1.0     NaN     NaN   
1        sub-10171   CONTROL   24      M   1.0  1.0  1.0     NaN     NaN   
2        sub-10189   CONTROL   49      M   1.0  NaN  1.0     NaN     NaN   
4        sub-10206   CONTROL   21      M   1.0  NaN  1.0     NaN     NaN   
5        sub-10217   CONTROL   33      F   1.0  NaN  1.0     NaN     NaN   
6        sub-10225   CONTROL   35      M   1.0  1.0  1.0     NaN     NaN   
7        sub-10227   CONTROL   31      F   1.0  NaN  1.0     NaN     NaN   
8        sub-10228   CONTROL   40      F   1.0  1.0  1.0     NaN     NaN   
9        sub-10235   CONTROL   22      M   1.0  NaN  1.0     NaN     NaN   
10       sub-10249   CONTROL   28      M   1.0  1.0  1.0     NaN     NaN   
11       sub-10269   CONTROL   38      M   1.0  NaN  1.0     NaN     NaN   
12       sub-10271   CONTROL   41      F   1.0  1.0  1.0     NaN     NaN   
13       sub-10273   CONTROL   30      F   1.0  1.0  1.0     NaN     NaN   
14       sub-10274   CONTROL   43      M   1.0  NaN  1.0     NaN     NaN   
15       sub-10280   CONTROL   27      M   1.0  1.0  1.0     NaN     NaN   
16       sub-10290   CONTROL   48      M   1.0  1.0  1.0     NaN     NaN   
17       sub-10292   CONTROL   49      M   1.0  1.0  1.0     NaN     NaN   
19       sub-10304   CONTROL   23      F   1.0  1.0  1.0     NaN     NaN   
21       sub-10321   CONTROL   34      M   1.0  1.0  1.0     NaN     NaN   
22       sub-10325   CONTROL   29      F   1.0  1.0  1.0     NaN     NaN   
23       sub-10329   CONTROL   22      M   1.0  1.0  1.0     NaN     NaN   
24       sub-10339   CONTROL   45      F   1.0  1.0  1.0     NaN     NaN   
25       sub-10340   CONTROL   32      M   1.0  1.0  1.0     NaN     NaN   
26       sub-10345   CONTROL   25      M   NaN  1.0  1.0     NaN     NaN   
27       sub-10347   CONTROL   25      F   1.0  1.0  1.0     NaN     NaN   
28       sub-10356   CONTROL   43      F   1.0  1.0  1.0     NaN     NaN   
29       sub-10361   CONTROL   42      F   1.0  1.0  1.0     NaN     NaN   
30       sub-10365   CONTROL   24      F   1.0  1.0  1.0     NaN     NaN   
31       sub-10376   CONTROL   42      F   1.0  1.0  1.0     NaN     NaN   
32       sub-10377   CONTROL   49      M   1.0  1.0  1.0     NaN     NaN   
..             ...       ...  ...    ...   ...  ...  ...     ...     ...   
132      sub-50006      SCHZ   44      F   1.0  1.0  1.0     NaN     NaN   
133      sub-50007      SCHZ   47      F   1.0  1.0  1.0     NaN     NaN   
135      sub-50010      SCHZ   41      M   1.0  1.0  1.0     NaN     NaN   
136      sub-50013      SCHZ   30      M   1.0  1.0  1.0     1.0     1.0   
137      sub-50014      SCHZ   22      M   1.0  1.0  1.0     1.0     1.0   
138      sub-50015      SCHZ   23      M   1.0  1.0  1.0     1.0     1.0   
139      sub-50016      SCHZ   49      M   1.0  1.0  1.0     1.0     1.0   
141      sub-50021      SCHZ   41      F   1.0  1.0  1.0     1.0     1.0   
143      sub-50023      SCHZ   49      F   1.0  1.0  1.0     1.0     1.0   
144      sub-50025      SCHZ   48      M   1.0  1.0  1.0     1.0     1.0   
150      sub-50035      SCHZ   29      F   1.0  1.0  1.0     1.0     1.0   
154      sub-50047      SCHZ   44      M   1.0  1.0  1.0     1.0     1.0   
155      sub-50048      SCHZ   43      M   1.0  1.0  1.0     1.0     1.0   
157      sub-50050      SCHZ   23      M   1.0  1.0  1.0     1.0     1.0   
159      sub-50052      SCHZ   45      M   1.0  1.0  1.0     1.0     1.0   
163      sub-50056      SCHZ   49      F   1.0  1.0  1.0     1.0     1.0   
164      sub-50058      SCHZ   38      F   1.0  1.0  1.0     1.0     1.0   
165      sub-50059      SCHZ   39      M   1.0  1.0  1.0     1.0     1.0   
167      sub-50061      SCHZ   48      M   1.0  1.0  1.0     1.0     1.0   
168      sub-50064      SCHZ   31      F   1.0  1.0  1.0     1.0     1.0   
169      sub-50066      SCHZ   33      M   1.0  1.0 

To ease the analysis and quicken the amount of time required to download the data, we're just going to use scans from 10 randomly sampled CONTROL and 10 SCHZ participants.

In [14]:
diagnosis_groups = participant_metadata.groupby('diagnosis')
filtered_participant_metadata = diagnosis_groups.apply(lambda x: x.sample(n = 10))
filtered_participant_metadata

participant_id diagnosis  age gender  bart  bht  dwi  pamenc  \
diagnosis                                                                    
CONTROL   25       sub-10340   CONTROL   32      M   1.0  1.0  1.0     NaN   
          8        sub-10228   CONTROL   40      F   1.0  1.0  1.0     NaN   
          125      sub-11131   CONTROL   38      F   1.0  1.0  1.0     1.0   
          64       sub-10678   CONTROL   24      F   1.0  1.0  1.0     1.0   
          126      sub-11142   CONTROL   24      M   1.0  1.0  1.0     1.0   
          93       sub-10963   CONTROL   45      F   1.0  1.0  1.0     1.0   
          51       sub-10527   CONTROL   24      M   1.0  1.0  1.0     1.0   
          19       sub-10304   CONTROL   23      F   1.0  1.0  1.0     NaN   
          31       sub-10376   CONTROL   42      F   1.0  1.0  1.0     NaN   
          82       sub-10871   CONTROL   41      M   1.0  1.0  1.0     1.0   
SCHZ      159      sub-50052      SCHZ   45      M   1.0  1.0  1.0     1.0   
          141      sub-50021      SCHZ   41      F   1.0  1.0  1.0     1.0   
          136      sub-50013      SCHZ   30      M   1.0  1.0  1.0     1.0   
          163      sub-50056      SCHZ   49      F   1.0  1.0  1.0     1.0   
          175      sub-50077      SCHZ   29      M   1.0  1.0  1.0     1.0   
          154      sub-50047      SCHZ   44      M   1.0  1.0  1.0     1.0   
          171      sub-50069      SCHZ   23      M   1.0  1.0  1.0     1.0   
          177      sub-50081      SCHZ   32      M   1.0  1.0  1.0     1.0   
          176      sub-50080      SCHZ   29      M   1.0  1.0  1.0     1.0   
          132      sub-50006      SCHZ   44      F   1.0  1.0  1.0     NaN   

               pamret  rest  scap  stopsignal  T1w  taskswitch  \
diagnosis                                                        
CONTROL   25      NaN   1.0   1.0         1.0  1.0         1.0   
          8       NaN   1.0   1.0         1.0  1.0         1.0   
          125     1.0   1.0   1.0         1.0  1.0         1.0   
          64      1.0   1.0   1.0         1.0  1.0         1.0   
          126     1.0   1.0   1.0         1.0  1.0         1.0   
          93      1.0   1.0   1.0         1.0  1.0         1.0   
          51      1.0   1.0   1.0         1.0  1.0         1.0   
          19      NaN   1.0   1.0         1.0  1.0         1.0   
          31      NaN   1.0   1.0         1.0  1.0         1.0   
          82      1.0   1.0   1.0         1.0  1.0         1.0   
SCHZ      159     1.0   1.0   1.0         1.0  1.0         1.0   
          141     1.0   1.0   1.0         1.0  1.0         1.0   
          136     1.0   1.0   1.0         1.0  1.0         1.0   
          163     1.0   1.0   1.0         1.0  1.0         1.0   
          175     1.0   1.0   1.0         1.0  1.0         1.0   
          154     1.0   1.0   1.0         1.0  1.0         1.0   
          171     1.0   1.0   1.0         1.0  1.0         1.0   
          177     1.0   1.0   1.0         1.0  1.0         1.0   
          176     1.0   1.0   1.0         1.0  1.0         1.0   
          132     NaN   1.0   1.0         1.0  1.0         1.0   

               ScannerSerialNumber ghost_NoGhost  
diagnosis                                         
CONTROL   25               35343.0      No_ghost  
          8                35343.0      No_ghost  
          125              35426.0      No_ghost  
          64               35343.0      No_ghost  
          126              35426.0      No_ghost  
          93               35343.0      No_ghost  
          51               35343.0      No_ghost  
          19               35343.0      No_ghost  
          31               35343.0      No_ghost  
          82               35343.0      No_ghost  
SCHZ      159              35426.0      No_ghost  
          141              35343.0      No_ghost  
          136              35343.0      No_ghost  
          163              35426.0      No_ghost  
          175              3

In [15]:
participant_list = filtered_participant_metadata.participant_id.tolist()
participant_list

['sub-10340',
 'sub-10228',
 'sub-11131',
 'sub-10678',
 'sub-11142',
 'sub-10963',
 'sub-10527',
 'sub-10304',
 'sub-10376',
 'sub-10871',
 'sub-50052',
 'sub-50021',
 'sub-50013',
 'sub-50056',
 'sub-50077',
 'sub-50047',
 'sub-50069',
 'sub-50081',
 'sub-50080',
 'sub-50006']

### Downloading Data

We've already randomly sampled 10 CONTROL and 10 SCHZ participants and placed the participant list in the `../download_list` text file. Let's download that data now.

In [16]:
# # download T1w scans
# !cat ../download_list | \
#   xargs -I '{}' aws s3 sync --no-sign-request \
#   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/{}/anat \
#   ../data/ds000030/{}/anat

# # download resting state fMRI scans
# !cat ../download_list | \
#   xargs -I '{}' aws s3 sync --no-sign-request \
#   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/{}/func \
#   ../data/ds000030/{}/func \
#   --exclude '*' \
#   --include '*task-rest_bold*'

# # download fmriprep preprocessed anat data
# !cat ../download_list | \
#   xargs -I '{}' aws s3 sync --no-sign-request \
#   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{}/anat \
#   ../data/ds000030/derivatives/fmriprep/{}/anat

# # download fmriprep preprocessed func data
# !cat ../download_list | \
#   xargs -I '{}' aws s3 sync --no-sign-request \
#   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{}/func \
#   ../data/ds000030/derivatives/fmriprep/{}/func \
#   --exclude '*' \
#   --include '*task-rest_bold*'

### Querying a BIDS Dataset

[pybids](https://bids-standard.github.io/pybids/) is a Python API for querying, summarizing and manipulating the BIDS folder structure.

In [17]:
from bids.layout import BIDSLayout

In [18]:
layout = BIDSLayout('../data/ds000030')

The pybids layout object lets you query your BIDS dataset according to a number of parameters by using a `get_*()` method.  
We can get a list of the subjects we've downloaded from the dataset.

In [19]:
layout.get_subjects()

['10171',
 '10292',
 '10365',
 '10438',
 '10565',
 '10788',
 '11106',
 '11108',
 '11122',
 '11131',
 '50010',
 '50035',
 '50047',
 '50048',
 '50052',
 '50067',
 '50075',
 '50077',
 '50081',
 '50083']

To get a list of all of the files, just use `get()`. 

In [20]:
layout.get()

[<BIDSFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/CHANGES'>,
 <BIDSFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/dataset_description.json'>,
 <BIDSDataFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/participants.tsv'>,
 <BIDSFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/README'>,
 <BIDSFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10171/anat/sub-10171_T1w.json'>,
 <BIDSImageFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz'>,
 <BIDSFile filename='/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10171/func/sub-10171_task-rest_bold.json'>,
 <BIDSImageFile filename='/mnt/tigr

There are many arguments we can use to filter down this list. Any BIDS-defined keyword can be passed on as a constraint. In `pybids`, these keywords are known as **entities**. For a complete list of possibilities:

In [21]:
layout.entities

{'subject': <bids.layout.models.Entity at 0x7fc31104cfd0>,
 'session': <bids.layout.models.Entity at 0x7fc3110ccdd8>,
 'task': <bids.layout.models.Entity at 0x7fc31104cf98>,
 'acquisition': <bids.layout.models.Entity at 0x7fc31104cc88>,
 'ce': <bids.layout.models.Entity at 0x7fc31104ceb8>,
 'reconstruction': <bids.layout.models.Entity at 0x7fc310fe33c8>,
 'dir': <bids.layout.models.Entity at 0x7fc310fe3eb8>,
 'run': <bids.layout.models.Entity at 0x7fc310fe3358>,
 'proc': <bids.layout.models.Entity at 0x7fc310fe3ac8>,
 'modality': <bids.layout.models.Entity at 0x7fc311005898>,
 'echo': <bids.layout.models.Entity at 0x7fc311005ba8>,
 'recording': <bids.layout.models.Entity at 0x7fc311005e10>,
 'suffix': <bids.layout.models.Entity at 0x7fc311005cf8>,
 'scans': <bids.layout.models.Entity at 0x7fc2b82b49e8>,
 'fmap': <bids.layout.models.Entity at 0x7fc3110056d8>,
 'datatype': <bids.layout.models.Entity at 0x7fc3110053c8>,
 'extension': <bids.layout.models.Entity at 0x7fc311005630>,
 'ImageT

For example, if we only want the file paths of all of our resting state fMRI scans,

In [22]:
layout.get(datatype='func', suffix='bold', task='rest', extensions=['.nii.gz'], return_type='file')

/scratch/mjoseph/.pyenv/versions/3.7.3/envs/fmri_venv/lib/python3.7/site-packages/bids/layout/layout.py:659: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


['/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10171/func/sub-10171_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10292/func/sub-10292_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10365/func/sub-10365_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10438/func/sub-10438_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10565/func/sub-10565_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10788/func/sub-10788_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-11106/func/sub-11106_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjo

**EXERCISE**: Retrieve the file paths of any scan where the subject is '10292' or '50081' and the `RepetitionTime` is 2 seconds.

In [23]:
layout.get(subject=['10292', '50081'], RepetitionTime=2, return_type='file')

['/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-10292/func/sub-10292_task-rest_bold.nii.gz',
 '/mnt/tigrlab/projects/mjoseph/tutorials/func/scwg2018_python_neuroimaging/data/ds000030/sub-50081/func/sub-50081_task-rest_bold.nii.gz']

Let's save the first file from our list of file paths to a variable and pull the metadata from its associated JSON file using the `get_metadata()` function.

In [24]:
fmri_file = layout.get(subject=['10292', '50081'], RepetitionTime=2, return_type='file')[0]
layout.get_metadata(fmri_file)

{'AccelNumReferenceLines': 24,
 'AccelerationFactorPE': 2,
 'AcquisitionMatrix': '64/0/0/64',
 'CogAtlasID': 'trm_4c8a834779883',
 'CogPOID': 'COGPO_00086',
 'DeviceSerialNumber': '35343',
 'EPIFactor': 128,
 'EchoTime': 0.03,
 'EchoTrainLength': 1,
 'EffectiveEchoSpacing': 0.000395,
 'FlipAngle': 90,
 'ImageType': 'ORIGINAL/PRIMARY/M/ND/MOSAIC',
 'ImagingFrequency': 123249635,
 'InPlanePhaseEncodingDirection': 'COL',
 'Instructions': "For the following part, you do not need to do anything other than keep your eyes open and remain relaxed. It is still important that you remain still and do not fall asleep, but you don't need to respond to anything. Any questions?",
 'MRAcquisitionType': '2D',
 'MagneticFieldStrength': 3,
 'ManufacturerModelName': 'TrioTim',
 'NumberOfAverages': 1,
 'NumberOfPhaseEncodingSteps': 63,
 'PatientPosition': 'HFS',
 'PercentPhaseFieldOfView': 100,
 'PercentSampling': 100,
 'PhaseEncodingDirection': 'j-',
 'PixelBandwidth': 1420,
 'ProtocolName': 'BOLD - RESTI

We can even collect the metadata for all of our fmri scans into a list and convert this into a dataframe.

In [25]:
metadata_list = []
all_fmri_files = layout.get(datatype='func', suffix='bold', return_type='file', extensions='.nii.gz')
for fmri_file in all_fmri_files:
    fmri_metadata = layout.get_metadata(fmri_file)
    metadata_list.append(fmri_metadata)
df = pd.DataFrame.from_records(metadata_list)
df

AccelNumReferenceLines  AccelerationFactorPE AcquisitionMatrix  \
0                       24                     2         64/0/0/64   
1                       24                     2         64/0/0/64   
2                       24                     2         64/0/0/64   
3                       24                     2         64/0/0/64   
4                       24                     2         64/0/0/64   
5                       24                     2         64/0/0/64   
6                       24                     2         64/0/0/64   
7                       24                     2         64/0/0/64   
8                       24                     2         64/0/0/64   
9                       24                     2         64/0/0/64   
10                      24                     2         64/0/0/64   
11                      24                     2         64/0/0/64   
12                      24                     2         64/0/0/64   
13                      24                     2         64/0/0/64   
14                      24                     2         64/0/0/64   
15                      24                     2         64/0/0/64   
16                      24                     2         64/0/0/64   
17                      24                     2         64/0/0/64   
18                      24                     2         64/0/0/64   
19                      24                     2         64/0/0/64   

           CogAtlasID      CogPOID DeviceSerialNumber  EPIFactor  EchoTime  \
0   trm_4c8a834779883  COGPO_00086              35343        128      0.03   
1   trm_4c8a834779883  COGPO_00086              35343        128      0.03   
2   trm_4c8a834779883  COGPO_00086              35343        128      0.03   
3   trm_4c8a834779883  COGPO_00086              35343        128      0.03   
4   trm_4c8a834779883  COGPO_00086              35343        128      0.03   
5   trm_4c8a834779883  COGPO_00086              35343        128      0.03   
6   trm_4c8a834779883  COGPO_00086              35426        128      0.03   
7   trm_4c8a834779883  COGPO_00086              35426        128      0.03   
8   trm_4c8a834779883  COGPO_00086              35426        128      0.03   
9   trm_4c8a834779883  COGPO_00086              35426        128      0.03   
10  trm_4c8a834779883  COGPO_00086              35343        128      0.03   
11  trm_4c8a834779883  COGPO_00086              35343        128      0.03   
12  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
13  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
14  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
15  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
16  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
17  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
18  trm_4c8a834779883  COGPO_00086              35426        128      0.03   
19  trm_4c8a834779883  COGPO_00086              35426        128      0.03   

    EchoTrainLength  EffectiveEchoSpacing  ...  SequenceName SequenceVariant  \
0                 1              0.000395  ...  *epfid2d1_64              SK   
1                 1              0.000395  ...  *epfid2d1_64              SK   
2                 1              0.000395  ...  *epfid2d1_64              SK   
3                 1              0.000395  ...  *epfid2d1_64              SK   
4                 1              0.000395  ...  *epfid2d1_64              SK   
5                 1              0.000395  ...  *epfid2d1_64              SK   
6                 1              0.000395  ...  *epfid2d1_64              SK   
7                 1              0.000395  ...  *epfid2d1_64              SK   
8                 1              0.000395  ...  *epfid2d1_64              SK   
9                 1              0.000395  ...  *epfid2d1_64              SK   
10            